<a href="https://colab.research.google.com/github/CidClayQuirino/api_als/blob/main/api_als.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# ============================================
# CONFIGURAÇÕES DA API
# ============================================

LOGIN_URL = "https://integration.s360web.com/api/login"
API_BASE = "https://integration.s360web.com/api/v1"

USER = "integracao.komatsu"
PASSWORD = "o8sqfl5h"

# Pasta local onde os CSV serão salvos
PASTA_SAIDA = "/content/s360_export/"


# ============================================
# AUTENTICAÇÃO
# ============================================

def autenticar():
    payload = {"login": USER, "password": PASSWORD}
    response = requests.post(LOGIN_URL, json=payload)

    if response.status_code != 200:
        raise Exception("Erro ao autenticar:", response.text)

    return response.json().get("token")


# ============================================
# CHAMADA GENÉRICA DE API
# ============================================

def chamar_api(method, endpoint, token, params=None, body=None):
    headers = {"Authorization": f"Bearer {token}"}
    url = f"{API_BASE}/{endpoint}"

    if method == "GET":
        response = requests.get(url, headers=headers, params=params)
    else:
        response = requests.post(url, headers=headers, json=body)

    if response.status_code != 200:
        print("Erro:", response.text)
        return None

    return response.json()


# ============================================
# COLETA DE EQUIPAMENTOS
# ============================================

def coletar_equipamentos(token):
    params = {"ativo": "true"}
    dados = chamar_api("GET", "equipamento/list", token, params=params)
    time.sleep(25)
    return dados


# ============================================
# COLETA DE AMOSTRAS (sampleResult/search)
# ============================================

def coletar_amostras(token):
# Ajuste solicitado: 365 dias antes até hoje
    data_fim = datetime.now().strftime("%Y-%m-%d")
    data_inicio = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

    body = {
        "readingStatus": None,   # busca tudo
        "markRead": True,        # marcar como lido
        "startDate": data_inicio,
        "endDate": data_fim,
        "page": 1,
        "pageSize": 500
    }

    dados = chamar_api("POST", "sampleResult/search", token, body=body)
    time.sleep(25)
    return dados


# ============================================
# SALVAR CSV
# ============================================

def salvar_csv(dados, nome):
    df = pd.DataFrame(dados)
    caminho = f"{PASTA_SAIDA}{nome}"
    df.to_csv(caminho, index=False, encoding="utf-8-sig")
    print(f"Arquivo salvo: {caminho}")


# ============================================
# EXECUÇÃO PRINCIPAL
# ============================================

def executar_rotina():
    print("Iniciando coleta:", datetime.now())

    token = autenticar()

    # Equipamentos
    equipamentos = coletar_equipamentos(token)
    if equipamentos:
        salvar_csv(equipamentos, f"equipamentos_{datetime.now().date()}.csv")

    # Amostras
    amostras = coletar_amostras(token)
    if amostras and "items" in amostras:
        salvar_csv(amostras["items"], f"amostras_{datetime.now().date()}.csv")

    print("Finalizado:", datetime.now())


# Criar pasta no Colab
import os
os.makedirs(PASTA_SAIDA, exist_ok=True)

# Executar manualmente
executar_rotina()